In [68]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import numpy as np
from io import BytesIO
from datetime import datetime, timedelta
import boto3
from src.config import *
from src.s3_utils import get_s3_object

pd.set_option('display.max_columns', None)

Pulling Raw Data

In [69]:
file_path = r"C:\Users\jklas\Downloads\gb_day_cycle_forecast.csv"
file = pd.read_csv(file_path)

cvp_agl_path = r"C:\Users\jklas\Downloads\cvp_agl.csv"
cvp_agl_uk = pd.read_csv(cvp_agl_path)

cvp_amzl_uk= get_s3_object(scheduling_bucket, "UK_CVP_Plan_All")
file.query('cycle=="sd"').head()

au_fcst_path = r"C:\Users\jklas\Downloads\au_fcst.csv"
au_fcst = pd.read_csv(au_fcst_path)

In [70]:
# missing SSD fcsts --> where to find?
# what is "over-night" cycle in AU/SG?

In [71]:
# unique_stations = file.query('cycle=="sd"')["business_type"].unique()
# unique_stations = file["cycle"].unique()
# unique_stations = file["business_type"].unique()
unique_stations = file.query('cycle=="sd"')
unique_stations.describe()

,forecast_weekday,forecast_horizon,model_mot_car_dpph,model_mot_lpv_dpph,model_mot_van_dpph,model_mot_bike_dpph,model_mot_car_spr,model_mot_lpv_spr,model_mot_van_spr,model_mot_bike_spr,forecast_car_vol,forecast_car_routes,forecast_car_hours,unbiased_forecast_car_vol,unbiased_forecast_car_routes,unbiased_forecast_car_hours,forecast_lpv_vol,forecast_lpv_routes,forecast_lpv_hours,unbiased_forecast_lpv_vol,unbiased_forecast_lpv_routes,unbiased_forecast_lpv_hours,forecast_van_vol,forecast_van_routes,forecast_van_hours,unbiased_forecast_van_vol,unbiased_forecast_van_routes,unbiased_forecast_van_hours,forecast_bike_vol,forecast_bike_routes,forecast_bike_hours,unbiased_forecast_bike_vol,unbiased_forecast_bike_routes,unbiased_forecast_bike_hours,mtp_spr,model_bias,forecast_vol,forecast_routes,forecast_hours,model_spr,model_dpph,unbiased_forecast_vol,unbiased_forecast_routes,unbiased_forecast_hours,unbiased_model_spr,unbiased_model_dpph,final_forecast_car_vol,final_forecast_car_routes,final_forecast_car_hours,final_forecast_lpv_vol,final_forecast_lpv_routes,final_forecast_lpv_hours,final_forecast_van_vol,final_forecast_van_routes,final_forecast_van_hours,final_forecast_bike_vol,final_forecast_bike_routes,final_forecast_bike_hours,final_forecast_vol,final_forecast_routes,final_forecast_hours
count,1820.00000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.0,1820.0,1820.0,1820.000000,1820.000000,1820.000000,1820.0,1820.0,1820.0,1820.000000,1820.000000,1820.000000,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.000000,1820.0,1820.0,1820.0,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.000000,1820.000000,1820.000000
mean,3.00000,12.500000,8.336271,8.495837,8.336271,8.336271,18.645161,19.403775,18.645161,18.645161,0.0,0.0,0.0,1039.266425,55.262711,125.437499,0.0,0.0,0.0,35.702147,1.862998,4.292825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1072.339533,0.0,0.0,0.0,18.834814,8.376162,1074.968572,57.125709,129.730324,18.652719,8.336839,1039.266425,55.262711,125.437499,35.702147,1.862998,4.292825,0.0,0.0,0.0,0.0,0.0,0.0,1074.968572,57.125709,129.730324
std,2.00055,7.502061,1.080902,0.981513,1.080902,1.080902,2.938262,2.572464,2.938262,2.938262,0.0,0.0,0.0,494.112468,28.027278,65.034696,0.0,0.0,0.0,17.550139,0.999839,2.343695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,533.680842,0.0,0.0,0.0,2.734515,1.026451,508.735486,28.869379,67.053898,2.930334,1.077720,494.112468,28.027278,65.034696,17.550139,0.999839,2.343695,0.0,0.0,0.0,0.0,0.0,0.0,508.735486,28.869379,67.053898
min,0.00000,0.000000,6.857891,6.917185,6.857891,6.857891,13.715782,15.936727,13.715782,13.715782,0.0,0.0,0.0,86.168710,6.249696,12.499392,0.0,0.0,0.0,0.792052,0.040868,0.094072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,15.023818,7.073922,86.960763,6.290847,12.594115,13.751682,6.869119,86.168710,6.249696,12.499392,0.792052,0.040868,0.094072,0.0,0.0,0.0,0.0,0.0,0.0,86.960763,6.290847,12.594115
25%,1.00000,6.000000,7.585412,8.005184,7.585412,7.585412,16.735606,17.513108,16.735606,16.735606,0.0,0.0,0.0,780.237931,34.311071,81.877355,0.0,0.0,0.0,21.235784,1.116151,2.483590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,816.417616,0.0,0.0,0.0,16.607912,7.847449,816.417616,35.920662,85.719582,16.709495,7.620285,780.237931,34.311071,81.877355,21.235784,1.116151,2.483590,0.0,0.0,0.0,0.0,0.0,0.0,816.417616,35.920662,85.719582
50%,3.00000,12.500000,8.130631,8.258216,8.130631,8.130631,17.980146,19.219778,17.980146,17.980146,0.0,0.0,0.0,1169.541369,55.387657,120.495627,0.0,0.0,0.0,40.960755,1.953997,4.553606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1214.419087,0.0,0.0,0.0,18.230506,8.108366,1214.419087,57.714012,125.453340,17.974146,8.127181,1169.541369,55.387657,120.

Maestro File Processing

In [72]:
file = file.copy()

file = file.rename(columns={
    "forecast_date": "date"
}) 

relevant_cols = [
    "station",
    "business_type",
    "cycle",
    "date",
    "forecast_car_vol",
    "forecast_lpv_vol"
]

cycle_map = {
    "cy1": "CYCLE_1",
    "cy2": "CYCLE_2",
    "sd_a": "CYCLE_SD_A",
    "sd_b": "CYCLE_SD_B",
    "sd_c": "CYCLE_SD_C",
    "adhoc": "AD_HOC",
    "day": "day",
    "brunch": "brunch",
    "lunch": "lunch",
    "bedtime": "bedtime",
    "dinner": "dinner",
    "sd": "sd"
}

file["cycle"] = file["cycle"].map(cycle_map, na_action=None)
file = file[file["cycle"] != "sd"]                                          # TODO: Removing cycle name "sd" as all forecasts in the Maestro file are zero for this cycle
file = file[relevant_cols]
file["date"] = pd.to_datetime(file["date"], dayfirst=False)
file["forecast_sum"] = file["forecast_car_vol"] + file["forecast_lpv_vol"]
file["car_share"] = np.where(file["forecast_sum"] == 0, 0, file["forecast_car_vol"] / file["forecast_sum"])
file["lpv_share"] = np.where(file["forecast_sum"] == 0, 0, file["forecast_lpv_vol"] / file["forecast_sum"])

file.head()

,station,business_type,cycle,date,forecast_car_vol,forecast_lpv_vol,forecast_sum,car_share,lpv_share
0,CC06,GSF,day,2025-04-27,15.576847,15.423153,31.0,0.502479,0.497521
1,CC06,GSF,day,2025-04-28,15.576847,15.423153,31.0,0.502479,0.497521
2,CC06,GSF,day,2025-04-29,14.069410,13.930590,28.0,0.502479,0.497521
3,CC06,GSF,day,2025-04-30,17.084284,16.915716,34.0,0.502479,0.497521
4,CC06,GSF,day,2025-05-01,16.079326,15.920674,32.0,0.502479,0.497521


CVP AMZL Data Processing

In [73]:
cvp_amzl_uk = cvp_amzl_uk.copy()

cvp_amzl_uk = cvp_amzl_uk.rename(columns={
    "ofd_date": "date",
    "ofd_volume": "snop_forecast"
}) 
cvp_amzl_uk["business_type"] = "AMZL"

relevant_cols = [
    "station",
    "business_type",    
    "cycle",            
    "date",    
    "snop_forecast"           
]

cvp_amzl_uk = cvp_amzl_uk[relevant_cols]
cvp_amzl_uk["date"] = pd.to_datetime(cvp_amzl_uk["date"], dayfirst=False)

cvp_amzl_uk["cycle"].value_counts()

cycle
CYCLE_1       8200
CYCLE_SD_C    5338
CYCLE_SD_A    4967
CYCLE_SD_D    1439
CYCLE_SD_B    1145
CYCLE_2        126
Name: count, dtype: int64

CVP AGL Data Processing

In [74]:
cvp_agl_uk = cvp_agl_uk.copy()

cvp_agl_uk = cvp_agl_uk.rename(columns={
    "ofd_date": "date",
    "forecast": "snop_forecast"
}) 

relevant_cols = [
    "station",
    "business_type",       
    "cycle",            
    "date",    
    "snop_forecast"           
]

cvp_agl_uk = cvp_agl_uk[relevant_cols]
cvp_agl_uk["date"] = pd.to_datetime(cvp_agl_uk["date"], dayfirst=False)
cvp_agl_uk["business_type"] = "GSF"
cvp_agl_uk["cycle"] = "day"

cvp_agl_uk.head()

,station,business_type,cycle,date,snop_forecast
0,UUK2,GSF,day,2020-06-15,496.0
1,ULI1,GSF,day,2020-07-17,303.0
2,ULI2,GSF,day,2020-10-31,496.0
3,CU09,GSF,day,2022-05-27,385.0
4,ULO5,GSF,day,2022-09-24,530.0


CVP SSD Data Processing

In [75]:
# TBD

CVP AU/SG Data

In [76]:
au_fcst = au_fcst.copy()

au_fcst = au_fcst.rename(columns={
    "forecast_delivery_date": "date",
    "pickup_location": "station",
    "dispatch_cycle": "cycle",
    "forecast_volume": "snop_forecast",
}) 

cycle_map = {
    "Cycle1": "CYCLE_1",
    "Cycle2": "CYCLE_2",
    "Same_day_1": "CYCLE_SD_A",
    "Same_day_2": "CYCLE_SD_B",
    "Over_Night": "Over_Night"
}

# au_fcst["cycle"] = au_fcst["cycle"].map(cycle_map, na_action=None)
au_fcst["date"] = pd.to_datetime(au_fcst["date"], dayfirst=False)
au_fcst = au_fcst[au_fcst["last_mile_carrier"]=="Flex"]

au_fcst["cycle"].unique()
# au_fcst["cycle"].unique()

array(['Cycle1', 'Cycle2', 'Over_Night', 'Same_day_1', 'Same_day_2'],
      dtype=object)

Joining all forecasts 

In [77]:
# Stack the CVP + AGL forecasts

forecasts = pd.concat([
    cvp_amzl_uk,
    cvp_agl_uk
], ignore_index=True)

forecasts.head()



,station,business_type,cycle,date,snop_forecast
0,DEH1,AMZL,CYCLE_1,2025-08-22,77299.589735
1,DCE1,AMZL,CYCLE_SD_C,2025-05-31,930.000000
2,DWR1,AMZL,CYCLE_SD_C,2025-08-02,1346.792243
3,DXN1,AMZL,CYCLE_SD_D,2025-05-29,545.931327
4,DRR1,AMZL,CYCLE_1,2025-08-16,24559.328728


Joining the Data

In [78]:
# Date Comparison

cvp_rng = forecasts["date"].sort_values(ascending=False).unique()
print(f"""
      CVP______
      Max: {cvp_rng[0]}
      Min: {cvp_rng[-1]}
      """)

maestro_rng = file["date"].sort_values(ascending=False).unique()
print(f"""
      Maestro______
      Max: {maestro_rng[0]}
      Min: {maestro_rng[-1]}
      """)


      CVP______
      Max: 2025-09-20 00:00:00
      Min: 2020-04-01 00:00:00
      

      Maestro______
      Max: 2025-10-25 00:00:00
      Min: 2025-04-27 00:00:00
      


In [91]:
# Join tables
join_cols = ["date", "station", "cycle", "business_type"]

df = file.merge(
    forecasts,
    on= join_cols,
    how="left",
    suffixes=["_maestro", "_snop"]
)

tolerance = 0.5
df["difference"] = df["forecast_sum"] - df["snop_forecast"]
df["difference"] = np.where(abs(df["difference"]) > tolerance, df["difference"], 0)

df["match"] = np.where((df["difference"] > 0), False, True)
df["match"] = np.where(df["snop_forecast"].isnull(), np.nan, df["match"])

filter = (df["date"] > datetime.now()) & (df["date"] < (datetime.now() + timedelta(weeks=2)))

df = df[filter]


df.head()

,station,business_type,cycle,date,forecast_car_vol,forecast_lpv_vol,forecast_sum,car_share,lpv_share,snop_forecast,difference,match
72,CC06,GSF,day,2025-06-03,15.576847,15.423153,31.0,0.502479,0.497521,30.0,1.0,0.0
73,CC06,GSF,day,2025-06-04,17.084284,16.915716,34.0,0.502479,0.497521,32.0,2.0,0.0
74,CC06,GSF,day,2025-06-05,16.581805,16.418195,33.0,0.502479,0.497521,31.0,2.0,0.0
75,CC06,GSF,day,2025-06-06,19.596679,19.403321,39.0,0.502479,0.497521,37.0,2.0,0.0
76,CC06,GSF,day,2025-06-07,18.591721,18.408279,37.0,0.502479,0.497521,37.0,0.0,1.0


Comparing the Data

In [ ]:
differences = df[df["match"] == 0]
matches = df[df["match"] == 1]
no_val = df[df["match"].isnull()]

print(f"""
    Rows overall:
    {df.shape[0]} ({df.query('business_type == "SSD"').shape[0]} SSD)
    ---------------------
    Count of Differences:
    {differences.shape[0]} ({differences.query('business_type == "SSD"').shape[0]} SSD)
    ---------------------
    Count of Matches:
    {matches.shape[0]} ({matches.query('business_type == "SSD"').shape[0]} SSD)
    ---------------------
    NaN values:
    {no_val.shape[0]} ({no_val.query('business_type == "SSD"').shape[0]} SSD)
    """
      )



    Rows overall:
    5628 (294 SSD)
    ---------------------
    Count of Differences:
    1638 (0 SSD)
    ---------------------
    Count of Matches:
    2450 (0 SSD)
    ---------------------
    NaN values:
    1540 (294 SSD)
    


In [81]:
differences.count()

station             1638
business_type       1638
cycle               1638
date                1638
forecast_car_vol    1638
forecast_lpv_vol    1638
forecast_sum        1638
car_share           1638
lpv_share           1638
snop_forecast       1638
difference          1638
match               1638
dtype: int64